# Model Prediction

RouteE models can be loaded from a large library of pre-trained models

In [1]:
import nrel.routee.powertrain as pt

In [5]:
pt.list_available_models()

['2016_TOYOTA_Camry_4cyl_2WD', '2016_TESLA_Model_S60_2WD']

In [6]:
camry = pt.load_pretrained_model("2016_TOYOTA_Camry_4cyl_2WD")

In [7]:
camry

========================================
Model Summary
--------------------
Vehicle description: 2016 Toyota Camry 4cyl 2WD
Powertrain type: ICE
Number of estimators: 2
========================================
Estimator Summary
--------------------
Feature: speed (mph)
Feature: grade (decimal_-1_to_1)
Distance: distance (miles)
Target: energy (gallons_gasoline)
Predicted Consumption: 31.581 (miles/gallons_gasoline)
========================================
Estimator Summary
--------------------
Feature: speed (mph)
Feature: grade (decimal_-1_to_1)
Feature: entry_angle (degrees)
Feature: exit_angle (degrees)
Distance: distance (miles)
Target: energy (gallons_gasoline)
Predicted Consumption: 31.455 (miles/gallons_gasoline)
========================================

In [8]:
import pandas as pd
links = pd.DataFrame({
    "speed": [10, 20, 30, 40, 50],
    "grade": [0.0, 0.0, -0.05, 0.05, 0.0],
    "distance": [1, 1, 1, 1, 1],
    "entry_angle": [0, 90, 0, 180, 0],
    "exit_angle": [90, 0, 180, 0, -90]
})

The pretrained Camry model has multiple estimators available for you to use depending on what features you might have available. You can see a summary of all the estimators in the model summary shown above but you can also just list which feature sets are available:

In [10]:
camry.estimators.keys()

dict_keys(['grade&speed', 'entry_angle&exit_angle&grade&speed'])

If we just pass in the links DataFrame without any other information, the model will assume we want to use all the feature and in this case will look for an internal estimator with the features ["speed", "grade", "entry_angle", "exit_angle"]

In [11]:
camry.predict(links)

,energy
0,0.053120
1,0.063292
2,0.018446
3,0.053990
4,0.033694


If we want to use a different estimator, we can tell the predict method to only use a subset of the features. In this case, we'll tell the model to only use speed & grade

In [12]:
camry.predict(links, feature_columns=["speed", "grade"])

,energy
0,0.058793
1,0.044744
2,0.018063
3,0.049676
4,0.033961


We can also apply some correction factors for effects that are not directly captured in the energy data the model was trained on. These include things like:
 - Ambient Temperature
 - Auxilary loads (i.e. Air Conditioners)

These are powertrain specific and can be included by passing the `apply_real_world_adjustment` paramter which is set to False by default.

In [13]:
camry.predict(links, apply_real_world_adjustment=True)

,energy
0,0.061938
1,0.073799
2,0.021508
3,0.062952
4,0.039287


Note that we see higher energy values when apply these factors

Lastly, we can take a look at the validation errors for each estimator that the model has.

In [14]:
camry.errors

=========================================
Feature Set ID:       grade&speed
Target:               energy
Link RMSE             0.002
Link Norm RMSE        0.938
Link Weighted RPD     0.398
Net Error             0.002
Actual Dist/Energy    31.652
Predicted Dist/Energy 31.581
Trip RPD              0.200
Trip Weighted RPD     0.120
Trip RMSE             0.012
Trip Norm RMSE        0.230
=========================================
Feature Set ID:       entry_angle&exit_angle&grade&speed
Target:               energy
Link RMSE             0.002
Link Norm RMSE        0.935
Link Weighted RPD     0.394
Net Error             0.006
Actual Dist/Energy    31.652
Predicted Dist/Energy 31.455
Trip RPD              0.198
Trip Weighted RPD     0.120
Trip RMSE             0.012
Trip Norm RMSE        0.228
=========================================